In [1]:
import numpy as np
import math as m
import itertools
import scipy as sp

def binary_decode(bytearray: list[int]):
    bytearray = np.flip(bytearray)
    return sum(bytearray * (2 ** np.array(list(enumerate(bytearray.flat)))[:,0]))

def binary_encode(ingest: list[int], num_bits: int):
    return np.floor_divide(ingest, (2 ** np.array(list(enumerate(range(0,(num_bits)))))[:,0])) % 2

def func(i):
    return m.tanh(i)

def float_func(i, precision, ancillary):
    num_bits = precision+ancillary
    val = i * (2**(precision))
    #print(val)
    bin_val = binary_encode(val,num_bits)
    return np.array(np.flip(bin_val))

def float_decomp(i, precision, ancillary):
    num_bits = precision+ancillary
    exponent = np.array([0]*ancillary + list(range(1,precision+1)))
    div = np.array([2]*(num_bits))
    div = div ** exponent
    div = np.array([1]*(num_bits)) / div
    return np.sum(div*i)


def build_basic_oracle(precision, ancillary):

    num_bits = precision+ancillary

    bin_inputs = np.array(list(itertools.product([0, 1], repeat=(precision))))

    bin_inputs = np.unique(np.pad(bin_inputs,(ancillary,0)), axis=0)

    inputs = np.apply_along_axis(float_decomp, 1, bin_inputs,precision,ancillary)

    index_inputs =  np.array(list(enumerate(inputs.tolist())))[:,0].astype(int)

    outputs = np.array([func(i) for i in inputs])

    bin_outputs = np.array([float_func(o,precision,ancillary) for o in outputs])

    index_outputs = np.array([binary_decode(bo) for bo in bin_outputs]).astype(int)

    output_vectors_matrix = sp.sparse.csr_matrix((np.ones(index_outputs.shape),(index_inputs,index_outputs)),(int(2**num_bits),(2**num_bits))).toarray()

    return output_vectors_matrix


def apply_operation(unitary, dec_input, precision, ancillary):

    num_bits = precision + ancillary

    input = float_func(dec_input, precision, ancillary)

    input_basis_state = np.zeros(2**num_bits)

    input_basis_state[int(binary_decode(input))] = 1

    output_container = input_basis_state.copy()

    for u in unitary:
        output_container = output_container @ u

    output_basis_state = output_container.copy()

    max_probs = np.argwhere(output_basis_state == np.amin(output_basis_state))

    top_predictions = np.array([float_decomp(np.flip(binary_encode(max_prob,num_bits)),precision,ancillary) for max_prob in max_probs])

    return np.min(top_predictions)

precision = 8
ancillary = 0
num_bits = precision + ancillary

basic_op = build_basic_oracle(precision,ancillary)

left,sigma,right = np.linalg.svd(basic_op)

n_sigma = np.round(sigma,decimals=0).nonzero()[0].size

c_sigma = np.diag(np.array([-1]*n_sigma + [1.j]*(sigma.size-n_sigma)))

num_tests = 90

test_inputs = np.random.rand(num_tests)

predictions = np.array([apply_operation([left,c_sigma,right],predict,precision,ancillary) for predict in test_inputs])

actuals = np.array([func(real) for real in (test_inputs)])

np.max(np.abs(predictions - actuals))


0.006636777104094341

In [2]:
#unitary

np.allclose(np.eye(len(left)), left.dot(left.T.conj()))


True

In [3]:
v = basic_op * -1.j

v = (v + v.T.conj()).astype(complex)

v = v + np.diag((np.sum(np.power(v,2),axis=1) == -4) * np.sqrt(5+0.j))

v = v + np.diag((np.sum(np.power(v,2),axis=1) == -3) * np.sqrt(4+0.j))

v = v + np.diag((np.sum(np.power(v,2),axis=1) == -2) * np.sqrt(3+0.j))

v = v + np.diag((np.sum(np.power(v,2),axis=1) == -1) * np.sqrt(2+0.j))

num_tests = 90

test_inputs = np.random.rand(num_tests)

predictions = np.array([apply_operation([v],predict,precision,ancillary) for predict in test_inputs])

actuals = np.array([func(real) for real in (test_inputs)])

np.max(np.abs(predictions - actuals))

0.007207435525769551

In [4]:
#skew hermitian

np.allclose(v.T.conj(),-v)

False

In [5]:
# hermitian

np.allclose(v.T.conj(),v)

True

In [6]:
#orthagonal

#np.allclose(v.T,np.linalg.inv(v))

In [7]:
#normal hermitian

np.allclose(v.T.conj()@v,v@v.T.conj())

True

In [8]:
#normal

print(np.unique(np.sum(np.power(v,2),axis=0)))
print(np.unique(np.sum(np.power(v,2),axis=1)))

[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[1.+0.j 1.+0.j 1.+0.j 1.+0.j]


In [9]:
# all real eigenvalues hermitian

eigenvalues,eigenvectors = np.linalg.eig(v)

real_eigenvalues = eigenvalues.real

np.allclose(np.abs(real_eigenvalues), np.abs(eigenvalues))


True

In [10]:
# all eigenvalues on complex unit circle "unitary"

np.allclose(np.abs(eigenvalues), 1)

False